In [1]:
from engine import SearchEngine
from evaluate_map import *
from optimizing import Optimize
# from ebrm import *

se = SearchEngine(
    entities_jsonl="robust04_entities_subset.jsonl",
    entities_offsets="robust04_entities_subset.jsonl.offsets.pkl"
)
topics = load_topics("Data/queriesROBUST.txt")

C:\Users\ariel\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[entity_df] building DF over subset: 100%|██████████| 1938/1938 [00:00<00:00, 14961.60it/s]


In [2]:
se.set_searcher(approach="qld", fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=340)

se.search_all_queries_hybrid_ebrm(
    topics=topics,
    k=1000,
    run_tag="rm3_best_plus_ebrm",
    output_file="run_hybrid_ebrm.txt",
    fb_docs=5,
    fb_entities=10,
    entity_weight=0.01,
    allowed_labels={"PERSON", "ORG", "GPE", "LOC"}
)


In [3]:
map = get_map_by_paths("Data/qrels_50_Queries", "run_hybrid_ebrm.txt")
map

0.15902270363585527

In [ ]:
# se.set_searcher(approach="qld", fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=340)
se.set_searcher(approach="bm25",  fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=340, k1=0.5, b=0.36)

# alphas = [0.82, 0.88, 0.93]
alphas = [0.88]
fb_entities_list = [15, 25, 35, 50]
# fb_entities_list = [15]

for alpha in alphas:
    for fb_entity in fb_entities_list:
        se.search_all_queries_ebrm_rerank(
            topics=topics,
            k=1000,
            output_file="run_rm3_plus_ebrm_rerank.txt",
            run_tag="rm3_plus_ebrm_rerank",
            fb_docs=5,
            fb_entities=fb_entity,
            alpha=alpha,  # start here
            # allowed_labels={"PERSON","ORG","GPE","LOC"}
            allowed_labels={"ORG", "PERSON"}
        )
        map = get_map_by_paths("Data/qrels_50_Queries", "run_rm3_plus_ebrm_rerank.txt")
        print(alpha, fb_entity, map)


0.88 15 0.2672179787101151


In [6]:
map = get_map_by_paths("Data/qrels_50_Queries", "run_rm3_plus_ebrm_rerank.txt")
map

0.2689824233052634

In [5]:
from rrf_fuse import rrf_fuse

rrfs = [10,20,30,40,50,60,70,80,90]
for rrf in rrfs:
    fused_path = rrf_fuse(
        run_a="run_bm25rm3_best.txt",
        run_b="run_rm3_plus_ebrm_rerank.txt",
        out_path="run_fused_rrf.txt",
        rrf_k=rrf,
        tag="rm3_ebrm_rrf"
    )
    map = get_map_by_paths("Data/qrels_50_Queries", "run_fused_rrf.txt")
    print(rrf, map)
    # fused_path


10 0.2719668213610911
20 0.2719677979216281
30 0.27196784309880007
40 0.2719679716059751
50 0.2719680431812228
60 0.2719711353271721
70 0.2719711353271721
80 0.27197120107355177
90 0.27197120107355177


In [9]:
map = get_map_by_paths("Data/qrels_50_Queries", "run_fused_rrf.txt")
map

0.27042408460761136